# Construir um modelo de classificação: Deliciosas Cozinhas Asiáticas e Indianas


## Classificadores de culinária 2

Nesta segunda lição sobre classificação, vamos explorar `mais formas` de classificar dados categóricos. Também aprenderemos sobre as implicações de escolher um classificador em vez de outro.

### [**Questionário pré-aula**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Pré-requisitos**

Assumimos que você completou as lições anteriores, já que vamos utilizar alguns conceitos aprendidos anteriormente.

Para esta lição, precisaremos dos seguintes pacotes:

-   `tidyverse`: O [tidyverse](https://www.tidyverse.org/) é uma [coleção de pacotes R](https://www.tidyverse.org/packages) projetada para tornar a ciência de dados mais rápida, fácil e divertida!

-   `tidymodels`: O [tidymodels](https://www.tidymodels.org/) é uma [estrutura de pacotes](https://www.tidymodels.org/packages/) para modelagem e aprendizagem de máquina.

-   `themis`: O [pacote themis](https://themis.tidymodels.org/) fornece passos extras de receitas para lidar com dados desbalanceados.

Você pode instalá-los com o seguinte comando:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternativamente, o script abaixo verifica se você possui os pacotes necessários para completar este módulo e os instala caso estejam ausentes.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Um mapa de classificação**

Na nossa [lição anterior](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), tentámos responder à pergunta: como escolher entre vários modelos? Em grande parte, isso depende das características dos dados e do tipo de problema que queremos resolver (por exemplo, classificação ou regressão?).

Anteriormente, aprendemos sobre as várias opções disponíveis para classificar dados utilizando o guia da Microsoft. O framework de Machine Learning em Python, Scikit-learn, oferece um guia semelhante, mas mais detalhado, que pode ajudar ainda mais a restringir os seus estimadores (outro termo para classificadores):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Dica: [visite este mapa online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) e clique ao longo do caminho para ler a documentação.
>
> O [site de referência do Tidymodels](https://www.tidymodels.org/find/parsnip/#models) também oferece uma excelente documentação sobre diferentes tipos de modelos.

### **O plano** 🗺️

Este mapa é muito útil quando se tem uma compreensão clara dos seus dados, pois pode 'percorrer' os seus caminhos até chegar a uma decisão:

-   Temos \>50 amostras

-   Queremos prever uma categoria

-   Temos dados rotulados

-   Temos menos de 100K amostras

-   ✨ Podemos escolher um Linear SVC

-   Se isso não funcionar, como temos dados numéricos

    -   Podemos tentar um ✨ KNeighbors Classifier

        -   Se isso não funcionar, tentar ✨ SVC e ✨ Ensemble Classifiers

Este é um caminho muito útil para seguir. Agora, vamos direto ao assunto utilizando o framework de modelagem [tidymodels](https://www.tidymodels.org/): uma coleção consistente e flexível de pacotes R desenvolvidos para incentivar boas práticas estatísticas 😊.

## 2. Dividir os dados e lidar com conjuntos de dados desequilibrados.

Nas nossas lições anteriores, aprendemos que havia um conjunto de ingredientes comuns entre as nossas cozinhas. Além disso, havia uma distribuição bastante desigual no número de cozinhas.

Vamos lidar com isso da seguinte forma:

-   Eliminando os ingredientes mais comuns que criam confusão entre cozinhas distintas, usando `dplyr::select()`.

-   Utilizando uma `recipe` que pré-processa os dados para prepará-los para a modelagem, aplicando um algoritmo de `over-sampling`.

Já vimos isso na lição anterior, então deve ser tranquilo 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Lidar com dados desequilibrados

Dados desequilibrados frequentemente têm efeitos negativos no desempenho do modelo. Muitos modelos funcionam melhor quando o número de observações é igual e, por isso, tendem a ter dificuldades com dados desequilibrados.

Existem principalmente duas formas de lidar com conjuntos de dados desequilibrados:

-   adicionar observações à classe minoritária: `Over-sampling`, por exemplo, utilizando um algoritmo SMOTE que gera novos exemplos sintéticos da classe minoritária com base nos vizinhos mais próximos desses casos.

-   remover observações da classe majoritária: `Under-sampling`

Na nossa lição anterior, demonstrámos como lidar com conjuntos de dados desequilibrados utilizando uma `recipe`. Uma recipe pode ser vista como um plano que descreve quais passos devem ser aplicados a um conjunto de dados para prepará-lo para análise. No nosso caso, queremos ter uma distribuição igual no número de nossas cozinhas para o nosso `training set`. Vamos começar!


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Agora estamos prontos para treinar modelos 👩‍💻👨‍💻!

## 3. Para além dos modelos de regressão multinomial

Na nossa lição anterior, analisámos os modelos de regressão multinomial. Vamos explorar alguns modelos mais flexíveis para classificação.

### Máquinas de Vetores de Suporte

No contexto de classificação, `Máquinas de Vetores de Suporte` são uma técnica de aprendizagem automática que tenta encontrar um *hiperplano* que "melhor" separa as classes. Vamos observar um exemplo simples:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ não separa as classes. H2~ separa, mas apenas com uma margem pequena. H3~ separa-as com a margem máxima.

#### Classificador Linear de Vetores de Suporte

O clustering de Vetores de Suporte (SVC) é uma técnica derivada da família de máquinas de Vetores de Suporte (SVM) em Aprendizagem Automática. No SVC, o hiperplano é escolhido para separar corretamente `a maioria` das observações de treino, mas `pode classificar incorretamente` algumas observações. Ao permitir que alguns pontos fiquem do lado errado, o SVM torna-se mais robusto a outliers, proporcionando assim uma melhor generalização para novos dados. O parâmetro que regula esta violação é chamado de `cost`, que tem um valor padrão de 1 (consulte `help("svm_poly")`).

Vamos criar um SVC linear definindo `degree = 1` num modelo SVM polinomial.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Agora que capturámos os passos de pré-processamento e a especificação do modelo num *workflow*, podemos avançar e treinar o SVC linear, avaliando os resultados ao mesmo tempo. Para métricas de desempenho, vamos criar um conjunto de métricas que avalie: `accuracy`, `sensitivity`, `Positive Predicted Value` e `F Measure`.

> `augment()` irá adicionar coluna(s) com as previsões aos dados fornecidos.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Máquina de Vetores de Suporte

A máquina de vetores de suporte (SVM) é uma extensão do classificador de vetores de suporte para acomodar uma fronteira não linear entre as classes. Essencialmente, as SVMs utilizam o *truque do kernel* para ampliar o espaço de características e assim se adaptar a relações não lineares entre as classes. Uma função kernel popular e extremamente flexível usada pelas SVMs é a *função de base radial.* Vamos ver como ela se comporta com os nossos dados.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Muito melhor 🤩!

> ✅ Por favor veja:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> para leitura adicional.

### Classificadores de Vizinhos Mais Próximos

O algoritmo *K*-nearest neighbor (KNN) prevê cada observação com base na sua *semelhança* com outras observações.

Vamos ajustar um ao nosso conjunto de dados.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Parece que este modelo não está a ter um desempenho muito bom. Provavelmente, alterar os argumentos do modelo (ver `help("nearest_neighbor")`) irá melhorar o desempenho do modelo. Certifique-se de experimentar.

> ✅ Por favor veja:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> para aprender mais sobre classificadores *K*-Nearest Neighbors.

### Classificadores em ensemble

Os algoritmos de ensemble funcionam combinando múltiplos estimadores base para produzir um modelo otimizado, seja através de:

`bagging`: aplicação de uma *função de média* a uma coleção de modelos base

`boosting`: construção de uma sequência de modelos que se complementam para melhorar o desempenho preditivo.

Vamos começar por experimentar um modelo Random Forest, que constrói uma grande coleção de árvores de decisão e depois aplica uma função de média para obter um modelo geral melhor.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Bom trabalho 👏!

Vamos também experimentar com um modelo de Árvore Reforçada.

Árvore Reforçada define um método de conjunto que cria uma série de árvores de decisão sequenciais, onde cada árvore depende dos resultados das árvores anteriores, numa tentativa de reduzir o erro de forma incremental. Este método foca-se nos pesos dos itens classificados incorretamente e ajusta o modelo do próximo classificador para corrigir.

Existem diferentes formas de ajustar este modelo (veja `help("boost_tree")`). Neste exemplo, vamos ajustar Árvores Reforçadas através do motor `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Por favor veja:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Explora o modelo AdaBoost, que é uma boa alternativa ao xgboost.
>
> para aprender mais sobre classificadores Ensemble.

## 4. Extra - comparando múltiplos modelos

Testámos vários modelos neste laboratório 🙌. Pode tornar-se cansativo ou trabalhoso criar muitos fluxos de trabalho a partir de diferentes conjuntos de pré-processadores e/ou especificações de modelos e depois calcular as métricas de desempenho uma a uma.

Vamos ver se conseguimos resolver isso criando uma função que ajusta uma lista de fluxos de trabalho no conjunto de treino e depois retorna as métricas de desempenho com base no conjunto de teste. Vamos usar `map()` e `map_dfr()` do pacote [purrr](https://purrr.tidyverse.org/) para aplicar funções a cada elemento de uma lista.

> As funções [`map()`](https://purrr.tidyverse.org/reference/map.html) permitem substituir muitos ciclos "for" por código que é mais conciso e mais fácil de ler. O melhor lugar para aprender sobre as funções [`map()`](https://purrr.tidyverse.org/reference/map.html) é o [capítulo de iteração](http://r4ds.had.co.nz/iteration.html) em R for Data Science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


O pacote [**workflowset**](https://workflowsets.tidymodels.org/) permite aos utilizadores criar e ajustar facilmente um grande número de modelos, mas foi principalmente concebido para funcionar com técnicas de reamostragem, como a `validação cruzada`, uma abordagem que ainda iremos abordar.

## **🚀Desafio**

Cada uma destas técnicas tem um grande número de parâmetros que podem ser ajustados, como por exemplo `cost` em SVMs, `neighbors` em KNN, `mtry` (Preditores Selecionados Aleatoriamente) em Random Forest.

Pesquise os parâmetros padrão de cada técnica e reflita sobre o que ajustar esses parâmetros significaria para a qualidade do modelo.

Para saber mais sobre um modelo específico e os seus parâmetros, utilize: `help("model")`, por exemplo, `help("rand_forest")`.

> Na prática, geralmente *estimamos* os *melhores valores* para estes parâmetros treinando vários modelos num `conjunto de dados simulado` e medindo o desempenho de todos esses modelos. Este processo é chamado de **otimização**.

### [**Questionário pós-aula**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Revisão & Estudo Individual**

Há muito jargão nestas lições, por isso reserve um momento para rever [esta lista](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) de terminologia útil!

#### AGRADECIMENTOS A:

[`Allison Horst`](https://twitter.com/allison_horst/) por criar as ilustrações incríveis que tornam o R mais acolhedor e envolvente. Encontre mais ilustrações na sua [galeria](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) e [Jen Looper](https://www.twitter.com/jenlooper) por criarem a versão original deste módulo em Python ♥️

Boas aprendizagens,

[Eric](https://twitter.com/ericntay), Embaixador Estudante Gold da Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Arte por @allison_horst</figcaption>



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante ter em conta que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
